In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

In [5]:
class PowerPlants(object):
    def __init__(self):
        self.database_file = "database.csv"
    
    def load_new_data_from_file(self, file_path: str):
        try: 
            #Read the csv files
            df = pd.read_csv(file_path) 
            
            #Convert all the country name into their full name 
            country_mapping = { 
                "FR": "France", 
                "GB": "Great Britain", 
                "Great Britain": "Great Britain", 
                "France": "France"
            }
            
            #Match the short and full name of the country
            country_col = "Country" if "Country" in df.columns else "Country "
            df["country"] = df[country_col].map(country_mapping).fillna(df[country_col]) 
            
            #Fill in missing Volume data with 0
            if "Volume" in df.columns:
                df["Volume"] = df["Volume"].fillna(0)
        
            #Add updatedby column if non-existent, and fill in with "Cody" if is missing 
            if "updatedby" not in df.columns:
                df["updatedby"] = "Cody"
           
            #Add updatetime column with current timestamp
            current_time = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S.%f")
            if "updatetime" not in df.columns:
                df["updatetime"] = current_time
            
            #Ensure all wanted columns exist
            required_columns = ["Date", "country", "SiteName", "Technology", "updatedby", "updatetime", "Volume"]
            
            #Reorder the columns
            df = df[["Date", "country", "SiteName", "Technology", "updatedby", "updatetime", "Volume"]]
            
            return df
            
        except Exception as e:
            print(f"Error in loading data from the csv files: {file_path}")
            print(f"Reasons: {e}")
            return pd.DataFrame()
    

    def save_new_data(self, input_data: pd.DataFrame):
        try:
            #Load the existing database.csv file
            df = pd.read_csv(self.database_file)
            
            if input_data.empty:
                print ("No new data to save.")
                return df
             
            #Add the three csv files into the existing database
            updated_df = pd.concat([df, input_data], ignore_index = True)
            
            #Save the updated data into the database.csv
            updated_df.to_csv(self.database_file, index = False) 
    
        except Exception as e:
            print(f"Error in saving new data to the database.csv file.")
            print(f"Reasons: {e}")
            
            
    def get_data_from_database(self):
        try:
            #Load the database.csv file with updated data 
            df = pd.read_csv(self.database_file) 
            
            #Convert updatetime to datetime format for proper sorting
            df["updatetime"] = pd.to_datetime(df["updatetime"]) 
            
            #Convert Date to datetime format for proper sorting
            if "Date" in df.columns:
                df["Date"] = pd.to_datetime(df["Date"], format = "%d/%m/%Y", errors = "coerce")
            
            #Sort the database by Technology first, then by updatetime (newest to oldest), and then by Date
            df = df.sort_values(["Technology", "updatetime", "Date"], ascending = [True, False, True])
            
            return df
        
        except Exception as e:
            print(f"Error in getting data from the database.csv file.")
            print(f"Reasons: {e}")
            return pd.DataFrame()
        

    def aggregate_data_to_monthly(self):
        try:             
            #Get the most recent data from database.csv file
            df = self.get_data_from_database()
            
            if df.empty:
                print("No data in the database.")
                return df

            #Filter to include Wind and Gas plants only
            df = df[df["Technology"].isin(["Gas", "Wind"])]

            #Convert Date to datetime format
            df["Date"] = pd.to_datetime(df["Date"], format = "%d/%m/%Y")

            #Create month column (first day of each month)
            df["month"] = df["Date"].dt.to_period("M").dt.start_time

            #Group by month and SiteName, calculate mean, min, max of Volume
            monthly_stats = df.groupby(["month", "SiteName"]).agg({"Volume": ["mean", "min", "max"]}).reset_index()

            #Rename the grouped columns
            monthly_stats.columns = ["date", "SiteName", "Mean", "Min", "Max"]

            #Pivot for Mean
            mean_pivot = monthly_stats.pivot(index = "date", columns = "SiteName", values = "Mean")
            mean_pivot.columns = [f"{col} Mean" for col in mean_pivot.columns]

            #Pivot for Min
            min_pivot = monthly_stats.pivot(index = "date", columns = "SiteName", values = "Min")
            min_pivot.columns = [f"{col} Min" for col in min_pivot.columns]

            #Pivot for Max
            max_pivot = monthly_stats.pivot(index = "date", columns = "SiteName", values = "Max")
            max_pivot.columns = [f"{col} Max" for col in max_pivot.columns]

            #Combine Mean, Min, Max
            result = pd.concat([mean_pivot, min_pivot, max_pivot], axis = 1).reset_index()

            #Sort columns alphabetically (except for date)
            date_col = result[["date"]]
            other_cols = result.drop("date", axis = 1)
            other_cols = other_cols.reindex(sorted(other_cols.columns), axis = 1)
            result = pd.concat([date_col, other_cols], axis = 1)

            #Format date column
            result["date"] = result["date"].dt.strftime("%Y-%m-%d")

            return result

        except Exception as e:
            print(f"Error in monthly aggregation.")
            print(f"Reasons: {e}")
            return pd.DataFrame()
        
        
    def aggregate_data_to_country(self):
        try:
            #Get the most recent data from database.csv file
            df = self.get_data_from_database()

            if df.empty:
                print("No data in the database.")
                return df

            #Group by country and technology, sum the volumes
            country_agg = df.groupby(["country", "Technology"]).agg({"Volume": "sum"})

            return country_agg

        except Exception as e:
            print(f"Error in country aggregation.")
            print(f"Reasons: {e}")
            return pd.DataFrame()

In [8]:
pp = PowerPlants()

new_data = pp.load_new_data_from_file("gas_fr_plants.csv")
pp.save_new_data(new_data)

new_data = pp.load_new_data_from_file("gas_plants.csv")
pp.save_new_data(new_data)

new_data = pp.load_new_data_from_file("wind_plants.csv")
pp.save_new_data(new_data)

In [9]:
pp = PowerPlants()

recent_data = pp.get_data_from_database()
print(recent_data)

           Date Country  Technology    SiteName  Volume        country date  \
5750 2024-01-01      NaN        Gas  Pembroke-1  6570.0  Great Britain  NaN   
6231 2024-01-01      NaN        Gas  Pembroke-2  8398.0  Great Britain  NaN   
5751 2024-01-02      NaN        Gas  Pembroke-1  8068.0  Great Britain  NaN   
6232 2024-01-02      NaN        Gas  Pembroke-2  8941.0  Great Britain  NaN   
5752 2024-01-03      NaN        Gas  Pembroke-1  7225.0  Great Britain  NaN   
...         ...      ...        ...         ...     ...            ...  ...   
5745        NaT      NaN        NaN         NaN     0.0            NaN  NaN   
5746        NaT      NaN        NaN         NaN     0.0            NaN  NaN   
5747        NaT      NaN        NaN         NaN     0.0            NaN  NaN   
5748        NaT      NaN        NaN         NaN     0.0            NaN  NaN   
5749        NaT      NaN        NaN         NaN     0.0            NaN  NaN   

     updatedby                 updatetime  
5750   

In [10]:
pp = PowerPlants()

monthly_data = pp.aggregate_data_to_monthly()
print(monthly_data)

          date  Blenod-5 Max  Blenod-5 Mean  Blenod-5 Min  Pembroke-1 Max  \
0   2024-01-01        6890.0    5198.806452        3295.0          8905.0   
1   2024-02-01        6895.0    4889.896552        3489.0          8970.0   
2   2024-03-01        6399.0    4752.774194        3122.0          8963.0   
3   2024-04-01        6970.0    4988.466667        3235.0          8941.0   
4   2024-05-01        6898.0    5012.032258        3059.0          8938.0   
5   2024-06-01        6870.0    4928.666667        3083.0          8978.0   
6   2024-07-01        6680.0    4933.677419        3115.0          8983.0   
7   2024-08-01        6973.0    5296.258065        3104.0          8977.0   
8   2024-09-01        6922.0    4858.066667        3110.0          8925.0   
9   2024-10-01        6973.0    4967.740741           0.0          8923.0   
10  2024-11-01        6942.0    4912.733333        3108.0          8975.0   
11  2024-12-01        6985.0    4981.000000        3087.0          8798.0   

In [11]:
pp = PowerPlants()

country_data = pp.aggregate_data_to_country()
print(country_data)

                                Volume
country       Technology              
France        Gas         7.138749e+06
Great Britain Gas         2.030437e+07
              Wind        1.426070e+06
